In [1]:
import pandas as pd
import numpy as np

unswap atoms

In [2]:
base = r"./"
filename = base + "data.out2"
fopen = open(filename,"r")
flines = fopen.readlines()
fopen.close()

In [3]:
bonds_map = list()
for _ in flines[144040:215940]:
    tmp = []
    atomi = int(_.split()[2])
    atomj = int(_.split()[3])
    tmp=[min(atomi,atomj),max(atomi,atomj)]
    bonds_map.append(tmp)
    
bonds_map.sort()

In [4]:
bond_table = pd.DataFrame(data = bonds_map)

In [5]:
bond_table.columns = ['i','j']

In [6]:
def add_edge(graph, u, v):
    if u not in graph:
        graph[u] = []
    if v not in graph:
        graph[v] = []
    graph[u].append(v)
    graph[v].append(u)

def dfs(node, graph, visited, component):
    # Stack for the nodes to visit
    stack = [node]
    while stack:
        current = stack.pop()
        if current not in visited:
            visited.add(current)
            component.append(current)
            for neighbor in graph[current]:
                if neighbor not in visited:
                    stack.append(neighbor)

def find_connected_components(connections):
    graph = {}
    # Building the graph
    for u, v in connections:
        add_edge(graph, u, v)
    
    visited = set()
    components = []

    # Finding all components
    for node in graph:
        if node not in visited:
            component = []
            dfs(node, graph, visited, component)
            components.append(component)
    
    return components

# Example usage
connections = bonds_map
#components = find_connected_components(connections)
#for i, component in enumerate(components):
    #print(f"Group {i + 1}: {component}")

reshuffled_chains = find_connected_components(connections)

In [7]:
swapedatom_df = pd.read_csv(base + "data.out2",skiprows=34,nrows=72000 ,sep="\s+",header=None)

In [8]:
swapedatom_df.columns=['atomid','molid','atype','q','x','y','z','nx','ny','nz']

In [9]:
swapedatom_df

,atomid,molid,atype,q,x,y,z,nx,ny,nz
0,12373,133,2,0,-20.847946,-21.194706,-20.583472,2,2,-2
1,44781,141,2,0,-21.192276,-21.701075,-21.513922,2,2,-1
2,12372,132,2,0,-21.744371,-20.982634,-20.822430,2,2,-2
3,44780,140,2,0,-20.283889,-21.466432,-21.396276,2,2,-1
4,44779,139,2,0,-19.663573,-21.604847,-20.619768,2,2,-1
...,...,...,...,...,...,...,...,...,...,...
71995,13949,269,2,0,18.256466,21.653581,20.763566,1,1,-2
71996,45095,455,2,0,19.364816,20.915448,21.273755,1,1,-2
71997,60456,696,2,0,20.149249,21.347806,20.590023,1,1,-2
71998,60022,262,2,0,21.743300,21.248155,20.994151,1,1,-2


In [10]:
for ichain, chain in enumerate(reshuffled_chains):
#    atoms.query('atomid==@chain')['type']=ichain+1
    tmp = swapedatom_df.query('atomid==@chain')
    index_arr = tmp.index
    swapedatom_df.loc[index_arr,'atype']=ichain+1
sort_atoms = swapedatom_df.sort_values(by=['atype','molid'])

In [11]:
sort_atoms

,atomid,molid,atype,q,x,y,z,nx,ny,nz
56636,1,1,1,0,15.499626,3.780977,-17.971203,1,2,-2
56634,2,2,1,0,14.623205,4.263755,-17.958017,1,2,-2
56655,2883,3,1,0,13.754028,4.693986,-18.139164,1,1,-2
56657,2884,4,1,0,13.554058,5.543854,-18.562827,1,1,-2
56679,2885,5,1,0,12.843875,6.112469,-18.921424,1,1,-2
...,...,...,...,...,...,...,...,...,...,...
7466,42476,716,100,0,-18.603106,-7.380184,18.717381,2,1,-2
7463,42477,717,100,0,-19.337360,-7.686314,18.227990,2,1,-2
7214,42478,718,100,0,-19.907872,-8.264914,17.757128,2,1,-2
7212,42479,719,100,0,-20.871989,-8.639038,17.427057,2,1,-2


backbone_clean

In [26]:
len_bb = 80
num_poly = 720
for index_atype in range(1,101):
    bb_list = [_ for _ in range(1,81)]
    pointer = 0;
    while pointer < len_bb-1:
        jdx = bb_list[pointer]
        jdx_1 = bb_list[pointer + 1]
        index_j = sort_atoms.query(f'atype == {index_atype} & molid == {jdx}').index
        index_k = sort_atoms.query(f'atype == {index_atype} & molid == {jdx_1}').index
        thisx = sort_atoms.loc[index_j,'x'].values[0]
        thatx = sort_atoms.loc[index_k,'x'].values[0]
        
        if thatx - thisx > 2:
            sort_atoms.loc[index_k,'x'] -= 21.934940186126006*2
        elif thatx - thisx < -2:
            sort_atoms.loc[index_k,'x'] += 21.934940186126006*2
        else:
            pass
        
        thisy = sort_atoms.loc[index_j,'y'].values[0]
        thaty = sort_atoms.loc[index_k,'y'].values[0]
        
        if thaty - thisy > 2:
            sort_atoms.loc[index_k,'y'] -= 21.934940186126006*2
        elif thaty - thisy < -2:
            sort_atoms.loc[index_k,'y'] += 21.934940186126006*2
        else:
            pass
        
        thisz = sort_atoms.loc[index_j,'z'].values[0]
        thatz = sort_atoms.loc[index_k,'z'].values[0]
        
        if thatz - thisz > 2:
            sort_atoms.loc[index_k,'z'] -= 21.934940186126006*2
        elif thatz - thisz < -2:
            sort_atoms.loc[index_k,'z'] += 21.934940186126006*2
        else:
            pass
        
        pointer += 1
        
    

In [28]:
###sidechain###
num_sc = 20
for index_atype in range(100):
    for index_of_sc in range(num_sc):
        jdx = 3 + index_of_sc*4 + index_atype*num_poly
        temp_connect1 = bond_table.query(f'i=={jdx}')['j'].tolist()
        temp_connect2 = bond_table.query(f'j=={jdx}')['i'].tolist()
        temp_connect = list(set(temp_connect1 + temp_connect2))
        atom1_list = [_ for _ in temp_connect if abs(_-jdx) > 2]
        for jdx_1 in atom1_list:
            jdx = 3 + index_of_sc*4 + index_atype*num_poly
            pointer = 0;
            tmp_mem = [jdx,jdx_1]
            while pointer < 33:
                index_j = sort_atoms.query(f'atomid=={jdx}').index
                index_k = sort_atoms.query(f'atomid=={jdx_1}').index
            
                thisx = sort_atoms.loc[index_j,'x'].values[0]
                thatx = sort_atoms.loc[index_k,'x'].values[0]
            
                if thatx - thisx > 3:
                    sort_atoms.loc[index_k,'x'] -= 21.934940186126006*2
                elif thatx - thisx < -3:
                    sort_atoms.loc[index_k,'x'] += 21.934940186126006*2
                else:
                    pass

                thisy = sort_atoms.loc[index_j,'y'].values[0]
                thaty = sort_atoms.loc[index_k,'y'].values[0]
                
                if thaty - thisy > 3:
                    sort_atoms.loc[index_k,'y'] -= 21.934940186126006*2
                elif thaty - thisy < -3:
                    sort_atoms.loc[index_k,'y'] += 21.934940186126006*2
                else:
                    pass
                

                thisz = sort_atoms.loc[index_j,'z'].values[0]
                thatz = sort_atoms.loc[index_k,'z'].values[0]
        
                if thatz - thisz > 3:
                    sort_atoms.loc[index_k,'z'] -= 21.934940186126006*2
                elif thatz - thisz < -3:
                    sort_atoms.loc[index_k,'z'] += 21.934940186126006*2
                else:
                    pass
                
                jdx = jdx_1
                temp_connect1 = bond_table.query(f'i=={jdx}')['j'].tolist()
                temp_connect2 = bond_table.query(f'j=={jdx}')['i'].tolist()
                temp_connect = list(set(temp_connect1 + temp_connect2))
                
                for atomj_1 in temp_connect:
                    if atomj_1 in tmp_mem:
                        pass
                    else:
                        jdx_1 = atomj_1
                pointer += 1
                tmp_mem.append(jdx_1)

In [ ]:
sort_atoms['molid'] = sort_atoms['atype']

In [32]:
sort_atoms

,atomid,molid,atype,q,x,y,z,nx,ny,nz
56636,1,1,1,0,15.499626,3.780977,-17.971203,1,2,-2
56634,2,2,1,0,14.623205,4.263755,-17.958017,1,2,-2
56655,2883,3,1,0,13.754028,4.693986,-18.139164,1,1,-2
56657,2884,4,1,0,13.554058,5.543854,-18.562827,1,1,-2
56679,2885,5,1,0,12.843875,6.112469,-18.921424,1,1,-2
...,...,...,...,...,...,...,...,...,...,...
7466,42476,716,100,0,-18.603106,-7.380184,-25.152500,2,1,-2
7463,42477,717,100,0,-19.337360,-7.686314,-25.641890,2,1,-2
7214,42478,718,100,0,-19.907872,-8.264914,-26.112752,2,1,-2
7212,42479,719,100,0,-20.871989,-8.639038,-26.442824,2,1,-2


newid_list = []
for molid in range(1,251):
    temp_df = sort_atoms.query(f'atype == {molid}')
    atmid_arr = temp_df["atomid"].tolist()
    for _ in atmid_arr:
        if _%480 ==0:
            newid_list.append(molid*480)
        else:
            newid_list.append(_%480 + molid*480 - 480)

In [33]:
newatype = []
for _ in sort_atoms['atomid'].tolist():
    if _%720 in range(1,81):
        newatype.append(1)
    else:
        newatype.append(2)
sort_atoms['atype'] = newatype

In [35]:
sort_atoms['atomid'] = np.arange(1,len(newatype)+1)


In [36]:
sort_atoms[['atomid','molid','atype','q','x','y','z']].to_csv(base + "atoms",index=None,header=None,sep=" ")

two filaments

In [ ]:
base2 = r"./"
atom_coord = pd.read_csv(base2 + "data.out_",header=None,skiprows=229,nrows=320000,sep="\s+")

In [ ]:
atom_coord.columns = ['atid','molid','atype','q','x','y','z','nx','ny','nz']

In [ ]:
sort_atom_coord = atom_coord.sort_values(by = 'atid')

In [ ]:
sort_atom_coord

In [ ]:
new_type = []
for mol in sort_atom_coord['molid'].tolist():
    if mol%100 <= 80 and mol%100 >0:
        new_type.append(2)
    else:
        new_type.append(1)

In [ ]:
sort_atom_coord['atype'] = new_type

In [ ]:
sort_atom_coord.to_csv(base2 + "/sort_atom_phasesep.dat",header=None,index=None,sep=" ")

DC accelerate

In [ ]:
atom_df = pd.read_csv("./data.out",skiprows=529,nrows=40000,sep="\s+",header=None)
atom_df.columns = ['atid','molid','atype','q','x','y','z','xu','yu','zu']

In [ ]:
atdf = atom_df.sort_values(by = 'atid')

In [ ]:
molid_arr = []
for i in range(250):
    temp_bb = [_+i*100 for _ in range(1,81)]
    temp_sc = [_+i*100 for _ in range(81,85)]*20
    temp = temp_bb + temp_sc
    molid_arr.extend(temp)

In [ ]:
atdf['molid'] = molid_arr

In [ ]:
atdf

In [ ]:
atdf.to_csv('./atom2',index=None,header=None,sep=" ")